# Separate Stacking Model

Here, we train a meta-learner that will best combine the predictions from the sub-models and ideally perform better than any single sub-model. The three sub-models are MODNet models trained on the PBE, HSE and experimental datasets.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy

In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

#### Loading the PBE, HSE and experimental datasets.

In [5]:
md_pbe = MODData.load('pbe_gap.zip')
md_pbe.df_targets.columns = ['gap']
md_hse = MODData.load('hse_gap.zip')
md_hse.df_targets.columns = ['gap']
md_exp = MODData.load('exp_gap_all')
md_exp.df_targets.columns = ['gap']
md_joint = MODData.load('exp_pbe_joint')
md_joint.df_targets = md_joint.df_targets.drop(columns=['pbe_gap', 'difference'])


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766

2021-06-14 17:51:27,046 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f902747b460> object, created with modnet version <=0.1.7
2021-06-14 17:51:30,781 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f902747b8e0> object, created with modnet version 0.1.8~develop
2021-06-14 17:51:35,674 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f90946f0310> object, created with modnet version <=0.1.7
2021-06-14 17:51:37,719 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f9094f127c0> object,

#### To get the df_structure

In [18]:
import pymatgen.ext.matproj

In [74]:
from pymatgen.ext.matproj import MPRester
import pandas as pd

with MPRester("v454AlrxL5Zg0cLAXW0") as mpr:
    index = md_pbe.df_structure.index
    array = np.empty(shape=md_pbe.df_structure.shape, dtype=pymatgen.core.structure.Structure)
    array = np.ndarray.flatten(array)
    for i in range(len(md_pbe.df_structure)):
        array[i] = mpr.get_structure_by_material_id(index[i])
    md_pbe.df_structure = pd.DataFrame(data=array, columns=["structure"])
    md_pbe.df_structure = md_pbe.df_structure.set_index(index)

/home/ucl/modl/heymans/anaconda3/envs/modnet/lib/python3.8/site-packages/pymatgen/ext/matproj.py:544: UserWarning: The calculation task mp-764266 is mapped to canonical mp-id mp-760156, so structure for mp-760156 returned. This is not an error, see documentation. If original task data for mp-764266 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn("The calculation task {} is mapped to canonical mp-id {}, "
/home/ucl/modl/heymans/anaconda3/envs/modnet/lib/python3.8/site-packages/pymatgen/ext/matproj.py:544: UserWarning: The calculation task mp-767290 is mapped to canonical mp-id mp-757267, so structure for mp-757267 returned. This is not an error, see documentation. If original task data for mp-767290 is required, use get_task_data(). To find the canonical mp-id from a task id use get_materials_id_from_task_id().
  warnings.warn("The calculation task {} is mapped to canonical mp-id {}, "
/home/ucl/modl/heyma

In [85]:
md_pbe.df_structure = pd.DataFrame(data=array, columns=["structure"])
md_pbe.df_structure = md_pbe.df_structure.set_index(index)

In [86]:
md_pbe.df_structure

,structure
id,
mp-647999,"[[6.41474125 3.84884475 3.84884475] Nb, [3.848..."
mp-764266,"[[1.56096903 5.3224765 2.0914864 ] Li, [4.788..."
mp-19019,"[[2.3082385 3.73247175 1.95323613] Ni, [2.321..."
mp-685999,"[[0. 6.93699153 2.46065822] Ag, [-1.45..."
mp-767290,"[[3.221996 3.89391826 0.82933335] Li, [1.747..."
...,...
mp-23292,[[ 2.51272300e+00 -1.16493568e+00 1.74069190e...
mp-1382,"[[-2.134272 3.201408 4.268544] Ce, [ 4.26854..."
mp-628705,"[[0. 8.20393607 4.48102764] Yb, [0. ..."


In [87]:
md_pbe.save('md-pbe')

2021-06-02 09:45:37,663 - modnet - INFO - Data successfully saved as md-pbe!


#### MODNet model trained on the PBE dataset

In [ ]:
# Complete model1:
fpath = 'md-pbe'
if os.path.exists(fpath):
    md_pbe = MODData.load(fpath)
else:
    md_pbe.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    md_pbe.save(fpath)
ga = FitGenetic(md_pbe)
model1 = ga.run(size_pop=20, num_generations=10, n_jobs=16)

In [ ]:
#model1.save('MODNetModel1')

In [6]:
model1 = MODNetModel.load('MODNetModel1')

2021-06-14 17:51:38,296 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f9094f191f0> object, created with modnet version 0.1.9


#### MODNet model trained on the HSE dataset

In [ ]:
# Complete model2:
fpath = 'md-hse_{}'
if os.path.exists(fpath):
    md_hse = MODData.load(fpath)
else:
    md_hse.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    md_hse.save(fpath)
ga = FitGenetic(md_hse)
model2 = ga.run(size_pop=20, num_generations=10, n_jobs=16)

2021-06-02 14:33:44,135 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f3e38fe4d60> object, created with modnet version 0.1.8~develop
2021-06-02 14:33:44,176 - modnet - INFO - Generation number 0
2021-06-02 14:33:49,784 - modnet - INFO - Multiprocessing on 16 cores. Total of 16 cores available.


100%|██████████| 100/100 [32:13<00:00, 19.33s/it]

2021-06-02 15:06:03,492 - modnet - INFO - Loss per individual: ind 0: 0.475 	ind 1: 0.559 	ind 2: 1.183 	ind 3: 0.452 	ind 4: 0.440 	ind 5: 0.487 	ind 6: 0.459 	ind 7: 0.445 	ind 8: 0.466 	ind 9: 0.447 	ind 10: 0.435 	ind 11: 0.465 	ind 12: 0.429 	ind 13: 0.743 	ind 14: 0.438 	ind 15: 0.499 	ind 16: 0.443 	ind 17: 0.630 	ind 18: 0.462 	ind 19: 0.438 	


2021-06-02 15:06:06,279 - modnet - INFO - Generation number 1
2021-06-02 15:06:14,072 - modnet - INFO - Multiprocessing on 16 cores. Total of 16 cores available.


100%|██████████| 100/100 [48:30<00:00, 29.10s/it]  

2021-06-02 15:54:44,675 - modnet - INFO - Loss per individual: ind 0: 0.422 	ind 1: 0.463 	ind 2: 0.479 	ind 3: 0.442 	ind 4: 14.083 	ind 5: 0.455 	ind 6: 0.475 	ind 7: 0.435 	ind 8: 0.497 	ind 9: 1.761 	ind 10: 0.419 	ind 11: 0.441 	ind 12: 0.486 	ind 13: 0.468 	ind 14: 0.488 	ind 15: 0.450 	ind 16: 0.759 	ind 17: 0.420 	ind 18: 0.703 	ind 19: 0.461 	


2021-06-02 15:56:03,962 - modnet - INFO - Generation number 2
2021-06-02 15:56:16,085 - modnet - INFO - Multiprocessing on 16 cores. Total of 16 cores available.


 96%|█████████▌| 96/100 [53:06<01:46, 26.53s/it]   

In [ ]:
#model2.save('MODNetModel2')

In [7]:
model2 = MODNetModel.load('MODNetModel2')

2021-06-14 17:51:38,607 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7f90285a3fd0> object, created with modnet version 0.1.9


### Separate Stacking Model

In [10]:
# stacked generalization with linear meta model on blobs dataset
import pandas as pd
from sklearn.metrics import accuracy_score

In [11]:
# load the saved models
def load_all_models():
    all_models = list()
    all_models = [model1, model2, model3]
    return all_models

In [12]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = pd.concat([model1.predict(inputX), model2.predict(inputX), model3.predict(inputX)], axis=1).to_numpy()
    return stackX

In [13]:
# Optional cell if you want to customize the model used in the fit_stacked_model() function (instead of the LinearRegression())
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss='mse',
                metrics=['accuracy'])
    return model

In [14]:
# fit a model based on the outputs from the ensemble members
from sklearn.linear_model import LinearRegression

def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = LinearRegression()
    model.fit(stackedX, inputy)
    return model

In [15]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [16]:
k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
         
        
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model3 = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    # load all models
    members = [model1, model2, model3]
    print('Loaded %d models' % len(members))
    
    # fit stacked model on test dataset
    model = fit_stacked_model(members, inputX=train, inputy=train.df_targets.to_numpy())

    # evaluate model on test set
    pred = stacked_prediction(members, model, test)
    pred = pd.DataFrame(pred)
    pred = pred.rename(columns={pred.columns[0]: 'exp_gap'})
    true = test.df_targets
    for j in range(len(pred)):
        pred = pred.rename(index={pred.index[j]: true.index[j]})
    error = pred-true
    error = error['exp_gap'].drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    print('MAE =')
    print(mae)
    maes[i] = mae

2021-06-14 17:54:27,009 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,037 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,053 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,079 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,095 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,121 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,137 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,160 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,176 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,205 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:54:27,569 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7f8f975c6160> object, created with modnet version 0.1.9
2021-06-14 17

100%|██████████| 100/100 [12:22<00:00,  7.42s/it]

2021-06-14 18:06:52,705 - modnet - INFO - Loss per individual: ind 0: 0.656 	ind 1: 0.632 	ind 2: 0.632 	ind 3: 0.582 	ind 4: 0.766 	ind 5: 0.678 	ind 6: 0.598 	ind 7: 0.761 	ind 8: 0.622 	ind 9: 0.594 	ind 10: 0.632 	ind 11: 0.657 	ind 12: 0.597 	ind 13: 0.623 	ind 14: 0.610 	ind 15: 0.625 	ind 16: 0.706 	ind 17: 0.593 	ind 18: 0.635 	ind 19: 0.601 	


2021-06-14 18:06:54,263 - modnet - INFO - Generation number 1
2021-06-14 18:06:56,787 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [11:33<00:00,  6.93s/it]

...

2021-06-15 01:05:16,786 - modnet - INFO - Loss per individual: ind 0: 0.610 	ind 1: 0.590 	ind 2: 0.619 	ind 3: 0.806 	ind 4: 0.595 	ind 5: 0.658 	ind 6: 0.625 	ind 7: 0.631 	ind 8: 0.571 	ind 9: 0.604 	ind 10: 0.650 	ind 11: 0.653 	ind 12: 0.613 	ind 13: 0.577 	ind 14: 0.604 	ind 15: 0.605 	ind 16: 0.582 	ind 17: 0.660 	ind 18: 0.592 	ind 19: 0.632 	


2021-06-15 01:05:18,015 - modnet - INFO - Generation number 6
2021-06-15 01:05:21,971 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [10:25<00:00,  6.25s/it]

2021-06-15 01:15:47,232 - modnet - INFO - Loss per individual: ind 0: 0.596 	ind 1: 0.661 	ind 2: 0.616 	ind 3: 0.583 	ind 4: 0.677 	ind 5: 0.601 	ind 6: 0.621 	ind 7: 0.733 	ind 8: 0.638 	ind 9: 0.652 	ind 10: 0.588 	ind 11: 0.617 	ind 12: 0.622 	ind 13: 0.727 	ind 14: 0.605 	ind 15: 0.693 	ind 16: 0.649 	ind 17: 0.632 	ind 18: 0.588 	ind 19: 0.584 	


2021-06-15 01:15:48,301 - modnet - INFO - Early stopping: same best model for 4 consecutive generations
Loaded 3 models
MAE =
0.34965069355762873


In [17]:
maes

array([0.37559103, 0.34188866, 0.2779486 , 0.31261524, 0.34965069])

In [18]:
maes.mean()

0.3315388449553086

### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: 4%